### Coverage, Serendipity, Ranking Metrics

In [1]:
import pandas as pd
from tqdm import tqdm
import json
import requests
pd.options.mode.chained_assignment = None
import numpy as np

In [2]:
#line_count = len(open("data/review.json").readlines())
line_count = len(open("data/business.json", encoding="utf8").readlines())
business_ids, cities, states, latitudes, longitudes, stars, review_counts = [], [], [], [], [], [], []
with open("data/business.json", encoding="utf8") as f:
    for line in tqdm(f, total=line_count):
        blob = json.loads(line)
        business_ids += [blob["business_id"]]
        cities += [blob["city"]]
        states += [blob["state"]]
        latitudes += [blob["latitude"]]
        longitudes += [blob["longitude"]]
        stars += [blob["stars"]]
        review_counts += [blob["review_count"]]
        
businesses = pd.DataFrame(
    {"business_id": business_ids, "city": cities, "state": states, "latitude": latitudes, "longitude": longitudes, "business_average_stars": stars, "business_review_counts": review_counts }
)

100%|██████████| 192609/192609 [00:03<00:00, 60140.06it/s]


In [3]:
#line_count = len(open("data/review.json").readlines())
line_count = len(open("data/user.json", encoding="utf8").readlines())
users, review_counts, elites, average_stars, friends = [], [], [], [], []
with open("data/user.json", encoding="utf8") as f:
    for line in tqdm(f, total=line_count):
        blob = json.loads(line)
        users += [blob["user_id"]]
        review_counts += [blob["review_count"]]
        elites += [blob["elite"]]
        average_stars += [blob["average_stars"]]
        friends += [blob["friends"]]
        
users = pd.DataFrame(
    {"user_id": users, "user_review_counts": review_counts,"elite": elites, "user_average_stars": average_stars, "friends": friends}
)

100%|██████████| 1637138/1637138 [00:31<00:00, 51269.38it/s]


In [4]:
ratings_holdout = pd.read_csv('data/ratings_sample_holdout.csv')

In [5]:
ratings_train = pd.read_csv('data/ratings_sample_train.csv')

In [6]:
ratings_val = pd.read_csv('data/ratings_sample_cv.csv')

In [7]:
def process(df):
    df = df.drop(df.columns[0], axis =1)
    df['date']  = pd.to_datetime(df['date'])
    df['week_day'] = df['date'].dt.weekday
    df['month'] = df['date'].dt.month
    df['hour'] = df['date'].dt.hour
    df = df.merge(users, on = 'user_id')
    df = df.merge(businesses, on = 'business_id')
    return df

In [8]:
ratings_train = process(ratings_train.copy())
ratings_holdout = process(ratings_holdout.copy())
ratings_val = process(ratings_val.copy())

We can calulate this after hyperparamter tuning. So we combine train + holdout

In [10]:
ratings_train_final = ratings_train.append(ratings_holdout)

First subsample a group of users that we will measure these metrics from

Methodology:
    We sample 5 users from each city where the user made the latest review.
    These cities must have at least 100 unique businesses
    These users must also have made a postive review(above their historical average)to those restaurants.
        1. We recommend 10 restaurants to each user
        2. We see if their latest restaurant makes it into the top 10 list (Ranking Metric)
        3. We see for those 10 x 5 recommendations, how many of them are distinct businesses (Coverage)
        4. We see for those top 10 recommendations, how many of them are restaurants they have not visited (Serendipity)
    
    Additionally, we measure what our ranking was for the latest restaurant that the user visited(Ranking Metric
    


Criteria: 

In [102]:
city = 'Strongsville'

In [104]:
tmp = ratings_train_final[(ratings_train_final['city'] ==city) &
                              (ratings_train_final['rating'] >ratings_train_final['user_average_stars'])]

In [107]:
five_users = np.random.choice(tmp['user_id'].unique(),5)

In [113]:
unique_city_businesses = ratings_train_final[['city','business_id']].drop_duplicates()
unique_cities = unique_city_businesses.groupby('city').count()['business_id']
unique_cities = unique_cities[unique_cities > 100]
out = pd.DataFrame()
for city in unique_cities.index:
    tmp = ratings_train_final[(ratings_train_final['city'] ==city) &
                              (ratings_train_final['rating'] >ratings_train_final['user_average_stars'])]
    if len(tmp['user_id'].unique())>4:
        
        ###this weird sampling technique is to ensure we dont' sample the same user twice in a same city
        five_users = np.random.choice(tmp['user_id'].unique(),5, replace = False)
        row = tmp[tmp['user_id'].isin(five_users)].groupby('user_id', group_keys=False).apply(lambda df: df.sample(1))
        out = out.append(row)

This is our subsample of users we will measure

In [115]:
all(out.groupby('city').count()['user_id']==5)

True

we merge in every business that user could have been to

In [159]:
predict_df = out[['user_id','city','state']]
predict_df = predict_df.merge(unique_city_businesses, on = 'city')

In [160]:
all(predict_df.groupby('city')['user_id'].nunique()==5)

True

Export this dataset. Everyone should make predictions on this dataset

In [25]:
predict_df.to_csv('data/metric_sample.csv')

In [161]:
predict_df['predictions'] = 2.5

Get top 10 recommendations for each user

In [162]:
top_10_recs = predict_df.groupby(['user_id','city'])['predictions'].nlargest(10).reset_index()

In [163]:
all(top_10_recs.groupby('city')['user_id'].count()==50)

True

In [165]:
cnt =0
serendipity = 0
for row in out.iterrows():
    row_values = row[1]
    top_10 = predict_df.loc[top_10_recs[top_10_recs['user_id'] == row_values['user_id']].level_2]['business_id']
    ###In top 10
    if row_values['business_id'] in top_10.values:
        cnt+=1
    user_history = ratings_train_final[ratings_train_final['user_id'] == row_values['user_id']]    
    been_there = [i for i in top_10.values if i in  user_history.business_id.values]
    serendipity += 1-len(been_there)/10

### 10% of users had their latest restaurant in our top 10 recommendations

In [166]:
cnt/len(out)

0.10285714285714286

### 97% of our recommendations were restaurant that user hasn't been

In [167]:
serendipity/len(out)

0.9719999999999989

In [168]:
predict_df = predict_df.reset_index()

In [169]:
predict_df.columns

Index(['index', 'user_id', 'city', 'state', 'business_id', 'predictions'], dtype='object')

In [172]:
top_10_recs.head(1)

,user_id,city,level_2,predictions
0,0BoVbu5ECOU89BF5h4nqgA,Henderson,97070,2.5


In [171]:
predict_df.head(1)

,index,user_id,city,state,business_id,predictions
0,0,1-xPzJk_ijBvY2J8Re1DIQ,Akron,OH,lMyCs9TssL9fbQR_Xiekcw,2.5


In [173]:
analysis_df = predict_df.merge(top_10_recs, left_on = ['user_id','city','index'], right_on = ['user_id','city','level_2'])

In [177]:
all(analysis_df.groupby('city')['business_id'].count() ==50)

True

### 20% of our recommendations were unique business ids in that city(Coverage)

In [183]:
(analysis_df.groupby('city')['business_id'].nunique()/50).values.mean()

0.19999999999999993

In [194]:
predict_df['rankings']=predict_df.groupby(['city','user_id'])['predictions'].rank("first",ascending = False)

In [207]:
running_rankings =0
for row in out.iterrows():
    row_values = row[1]
    user_recs = predict_df[(predict_df['user_id']==row_values['user_id'])
                        &(predict_df['city']==row_values['city'])
                         & (predict_df['business_id']==row_values['business_id'])
                          ]
    assert len(user_recs)==1
    running_rankings += user_recs['rankings'].sum()

### Average ranking of the latest restaurant that the user visited 

In [208]:
running_rankings / len(out)

498.25142857142856